In [13]:
%cd C:/Users/Mathieu/Desktop/Projets/Benter

C:\Users\Mathieu\Desktop\Projets\Benter


In [14]:
import datetime as dt
import numpy as np
import pandas as pd

from scripts.scrape_pmu_data import get_pmu_api_url, execute_get_query
from scripts.generate_pmu_data import get_num_pmu_enjeu, convert_queried_data_to_race_horse_df

from utils import features
from utils import import_data
from utils import model as utils_model
from utils import wagering_stategy
from constants import PMU_BETTINGS

In [15]:
TIMEZONE = 'Europe/Paris'
import pytz
tz = pytz.timezone('Europe/Paris')

In [16]:
date= dt.date.today()
#date = date - dt.timedelta(days=1)
code_pari = "E_SIMPLE_GAGNANT"
capital_fraction=0.01

In [17]:
def get_num_pmu_citation_enjeu(citations: dict, pari_type: str):
    if "listeCitations" not in citations:
        return None
    citations_ = [citation for citation in citations['listeCitations'] if citation['typePari']==code_pari]
    assert len(citations_) <= 1
    if not citations_:
        return None
    citation = citations_[0]
    if "participants" not in citation:
        return None

    return {
        part['numPmu']: part['citations'][0]['enjeu']
        for part in citation['participants']
    }

In [18]:
programme = execute_get_query(url=get_pmu_api_url(url_name="PROGRAMME", date=date))

In [19]:
historical_race_horse_df = import_data.load_featured_data(source="PMU")

utils_model.load_shared_layers(trainable=False)
_,_,_,median_na_feature_filler=import_data.get_splitted_featured_data(source='PMU')
track_take = [betting for betting in PMU_BETTINGS if betting.name == code_pari][0][
        1
    ]

In [38]:
%%time

dt_now = tz.localize(dt.datetime.now())

race_times = {}
for reunion in programme['programme']['reunions']:
    for course in reunion['courses']:
        race_times[(date, course["numReunion"], course["numOrdre"])]=dt.datetime.fromtimestamp(
                    course["heureDepart"] / 1000,
                    tz=dt.timezone(dt.timedelta(milliseconds=course["timezoneOffset"])),
                )-dt_now

coming_races = {(date, r_i, c_i):time_to_race for (date, r_i, c_i), time_to_race in race_times.items() if time_to_race.total_seconds()>0}
next_date, r_i, c_i = min(coming_races, key=coming_races.get)
print(f"Time to next race: {coming_races[(next_date, r_i, c_i)]}, date {date}, R{r_i} C{c_i}")

courses_ = [course for reunion in programme['programme']['reunions'] for course in reunion['courses'] if course["numReunion"]==r_i and course["numOrdre"]==c_i]
assert len(courses_)==1
course=courses_[0]

course_race_datetime = dt.datetime.fromtimestamp(
    course["heureDepart"] / 1000,
    tz=dt.timezone(dt.timedelta(milliseconds=course["timezoneOffset"])),
)

participants_ = execute_get_query(url=get_pmu_api_url(url_name="PARTICIPANTS", date=date, r_i=r_i, c_i=c_i))
assert "participants" in participants_

participants_ = participants_["participants"]
participants = [
    {k: v for k, v in part.items() if not isinstance(v, dict)}
    for part in participants_
]
course_incidents = course["incidents"] if "incidents" in course else []
incident_nums = {
    num_part
    for incident in course_incidents
    for num_part in incident["numeroParticipants"]
}

combinaisons =  execute_get_query(url=get_pmu_api_url(url_name="COMBINAISONS", date=date, r_i=r_i, c_i=c_i)+"?specialisation=INTERNET")
citations =  execute_get_query(url=get_pmu_api_url(url_name="CITATIONS", date=date, r_i=r_i, c_i=c_i)+"?specialisation=INTERNET")
num_pmu_enjeu = get_num_pmu_enjeu(
    combinaisons=combinaisons, pari_type="E_SIMPLE_GAGNANT"
)

num_pmu_citation_enjeu =get_num_pmu_citation_enjeu(citations=citations, pari_type="E_SIMPLE_GAGNANT")

for part, part_ in zip(participants, participants_):
    # Other dict key found {'commentaireApresCourse',
    #  'dernierRapportDirect',
    #  'dernierRapportReference',
    #  'distanceChevalPrecedent',
    #  'gainsParticipant', # added here
    #  'robe'}
    if "gainsParticipant" in part_:
        part.update(part_["gainsParticipant"])
    part["n_reunion"] = r_i
    part["n_course"] = c_i
    part["date"] = date
    part["race_datetime"] = course_race_datetime
    part["in_incident"] = part["numPmu"] in incident_nums
    part["incident_type"] = (
        None
        if part["numPmu"] not in incident_nums
        else [
            incident["type"]
            for incident in course_incidents
            if part["numPmu"] in incident["numeroParticipants"]
        ][0]
    )
    part["totalEnjeu"] = (
        None
        if num_pmu_citation_enjeu is None
        else num_pmu_citation_enjeu.get(part["numPmu"], None)
    )

race_df = pd.DataFrame.from_records(participants)

race_df = convert_queried_data_to_race_horse_df(queried_race_horse_df=race_df, historical_race_horse_df=historical_race_horse_df)

race_df=features.append_features(race_horse_df=race_df, historical_race_horse_df=historical_race_horse_df)
race_df=race_df[race_df['statut']!="NON_PARTANT"]

race_df['horse_place']=np.nan

x_race, y_race, odds_race = import_data.extract_x_y_odds(race_df=race_df, y_format='first_position', median_na_feature_filler=median_na_feature_filler, ignore_y=True)
bettings = wagering_stategy.race_betting_best_expected_return(x_race=x_race, odds_race=odds_race, track_take=track_take, capital_fraction=capital_fraction)

race_df['betting']=bettings

print(race_df[['horse_number','horse_name','betting']].set_index('horse_number'))

Time to next race: 0:01:15.562255, date 2020-04-27, R7 C5


 42%|█████████████████████████████████████████████████████████████████████████████████▎                                                                                                                 | 5/12 [00:01<00:01,  4.00it/s]C:\Users\Mathieu\Anaconda3\envs\deeplearning\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Mathieu\Anaconda3\envs\deeplearning\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  4.80it/s]


                     horse_name  betting
horse_number                            
1                    LOS SUENOS     0.00
2                       MIZONEY     0.00
3               MAYDAY'S PARADE     0.00
4                   SEBASTIANNA     0.00
5                  SO SO SAUCEY     0.00
6                LUCKY LIZA ANN     0.01
7             GOSPEL TRUMPY ANN     0.00
8                   LANDRY KYLE     0.00
9                CHIRON ECLIPSE     0.00
10                RAT A TAT TAT     0.00
11              POSSE GONE WILD     0.00
12            PULPIT'S PRINCESS     0.00
Wall time: 6.94 s


In [35]:
race_df[['horse_name', 'horse_number', 'statut', 'totalEnjeu', 'betting']].set_index('horse_number')

,horse_name,statut,totalEnjeu,betting
horse_number,,,,
1,LOS SUENOS,PARTANT,46215,0.00
2,MIZONEY,PARTANT,23220,0.00
3,MAYDAY'S PARADE,PARTANT,15850,0.00
4,SEBASTIANNA,PARTANT,17030,0.00
5,SO SO SAUCEY,PARTANT,11190,0.00
6,LUCKY LIZA ANN,PARTANT,12825,0.01
7,GOSPEL TRUMPY ANN,PARTANT,11625,0.00
8,LANDRY KYLE,PARTANT,55890,0.00
9,CHIRON ECLIPSE,PARTANT,28450,0.00


In [25]:
num_pmu_enjeu

{1: 29095, 2: 91965, 3: 36720, 4: 152780, 5: 221993, 6: 20740, 7: 121795}

In [26]:
num_pmu_citation_enjeu

{1: 29095, 2: 91965, 3: 36720, 4: 152780, 5: 221993, 6: 20740, 7: 121795}

In [64]:
STATUS = set()
for reunion in programme['programme']['reunions']:
    for course in reunion['courses']:
        STATUS.add(course['statut'])

In [65]:
STATUS

{'FIN_COURSE', 'PROGRAMMEE', 'ROUGE_AUX_PARTANTS'}

In [31]:
course['heureDepart']

1587933060000

In [9]:
course_datetime = dt.datetime.fromtimestamp(
                    course["heureDepart"] / 1000,
                    tz=dt.timezone(dt.timedelta(milliseconds=course["timezoneOffset"])),
                )

In [45]:
race_times = {}
for reunion in programme['programme']['reunions']:
    for course in reunion['courses']:
        race_times[(date, course["numReunion"], course["numOrdre"])]=dt.datetime.fromtimestamp(
                    course["heureDepart"] / 1000,
                    tz=dt.timezone(dt.timedelta(milliseconds=course["timezoneOffset"])),
                )-dt_now

coming_races = {(date, r_i, c_i):time_to_race for (date, r_i, c_i), time_to_race in race_times.items() if time_to_race.total_seconds()>0}
next_date, next_r_i, next_c_i = min(coming_races, key=coming_races.get)

In [37]:
a=(dt.datetime.fromtimestamp(
                    course["heureDepart"] / 1000,
                    tz=dt.timezone(dt.timedelta(milliseconds=course["timezoneOffset"])),
                )-dt_now)


In [40]:
a.total_seconds()

21379.181815

In [21]:
-list(race_times.values())[0]

datetime.timedelta(0, 10081, 737748)

In [22]:
list(race_times.values())[0]

datetime.datetime(2020, 4, 26, 13, 45, tzinfo=datetime.timezone(datetime.timedelta(0, 7200)))

In [ ]:
# Exploring my winning race R1C11 27/04/2020 à 21h49

In [39]:
date

datetime.date(2020, 4, 27)

In [41]:
r_i=1
c_i=11

In [42]:
courses_ = [course for reunion in programme['programme']['reunions'] for course in reunion['courses'] if course["numReunion"]==r_i and course["numOrdre"]==c_i]
assert len(courses_)==1
course=courses_[0]

course_race_datetime = dt.datetime.fromtimestamp(
    course["heureDepart"] / 1000,
    tz=dt.timezone(dt.timedelta(milliseconds=course["timezoneOffset"])),
)

participants_ = execute_get_query(url=get_pmu_api_url(url_name="PARTICIPANTS", date=date, r_i=r_i, c_i=c_i))
assert "participants" in participants_

participants_ = participants_["participants"]
participants = [
    {k: v for k, v in part.items() if not isinstance(v, dict)}
    for part in participants_
]
course_incidents = course["incidents"] if "incidents" in course else []
incident_nums = {
    num_part
    for incident in course_incidents
    for num_part in incident["numeroParticipants"]
}

combinaisons =  execute_get_query(url=get_pmu_api_url(url_name="COMBINAISONS", date=date, r_i=r_i, c_i=c_i)+"?specialisation=INTERNET")
citations =  execute_get_query(url=get_pmu_api_url(url_name="CITATIONS", date=date, r_i=r_i, c_i=c_i)+"?specialisation=INTERNET")
num_pmu_enjeu = get_num_pmu_enjeu(
    combinaisons=combinaisons, pari_type="E_SIMPLE_GAGNANT"
)

num_pmu_citation_enjeu =get_num_pmu_citation_enjeu(citations=citations, pari_type="E_SIMPLE_GAGNANT")

for part, part_ in zip(participants, participants_):
    # Other dict key found {'commentaireApresCourse',
    #  'dernierRapportDirect',
    #  'dernierRapportReference',
    #  'distanceChevalPrecedent',
    #  'gainsParticipant', # added here
    #  'robe'}
    if "gainsParticipant" in part_:
        part.update(part_["gainsParticipant"])
    part["n_reunion"] = r_i
    part["n_course"] = c_i
    part["date"] = date
    part["race_datetime"] = course_race_datetime
    part["in_incident"] = part["numPmu"] in incident_nums
    part["incident_type"] = (
        None
        if part["numPmu"] not in incident_nums
        else [
            incident["type"]
            for incident in course_incidents
            if part["numPmu"] in incident["numeroParticipants"]
        ][0]
    )
    part["totalEnjeu"] = (
        None
        if num_pmu_citation_enjeu is None
        else num_pmu_citation_enjeu.get(part["numPmu"], None)
    )

race_df = pd.DataFrame.from_records(participants)

race_df = convert_queried_data_to_race_horse_df(queried_race_horse_df=race_df, historical_race_horse_df=historical_race_horse_df)

race_df=features.append_features(race_horse_df=race_df, historical_race_horse_df=historical_race_horse_df)
race_df=race_df[race_df['statut']!="NON_PARTANT"]

race_df['horse_place']=np.nan

x_race, y_race, odds_race = import_data.extract_x_y_odds(race_df=race_df, y_format='first_position', median_na_feature_filler=median_na_feature_filler, ignore_y=True)
bettings = wagering_stategy.race_betting_best_expected_return(x_race=x_race, odds_race=odds_race, track_take=track_take, capital_fraction=capital_fraction)

race_df['betting']=bettings

print(race_df[['horse_number','horse_name','betting']].set_index('horse_number'))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:02<00:00,  4.53it/s]


                    horse_name  betting
horse_number                           
1              MUSIC'S CLASSIC     0.00
2                  RED CROSSER     0.00
3                AGNES LIXARVE     0.00
4                    EASY COPI     0.00
5              LIVI ON THE RUN     0.00
6             KING SIR RAYMOND     0.00
7             STEPPING FERRARI     0.00
8               GLOBAL WILDCAT     0.01
9                 WISS ZORNITA     0.00
10              ST NO MORE WAR     0.00
11                 MIDAS METAN     0.00


In [43]:
odds_race

array([17.12823498, 67.50344097, 21.85392328,  9.84436038,  4.47357373,
       22.84924143,  4.48653308, 40.37382586, 58.18129278,  6.31288678,
       11.24710033])

In [52]:
n_horses = x_race.shape[0]

model = utils_model.create_model(n_horses=n_horses, y_format="probabilities")
y_hat_race = model.predict(x=np.expand_dims(x_race, axis=0))[0, :]

expected_return_race = y_hat_race * odds_race * (1 - track_take)

race_df['y_hat']=y_hat_race
race_df['expected_return']=expected_return_race

In [53]:
race_df['expected_return']

0     0.972621
1     1.544366
2     2.591441
3     1.001817
4     0.655754
5     1.273744
6     0.472859
7     2.728590
8     1.203117
9     0.103905
10    1.670707
Name: expected_return, dtype: float64

In [60]:
df=race_df[['horse_number', 'horse_name', 'totalEnjeu', 'odds', 'y_hat', 'expected_return']].set_index('horse_number')
df

,horse_name,totalEnjeu,odds,y_hat,expected_return
horse_number,,,,,
1,MUSIC'S CLASSIC,44668,17.128235,0.065952,0.972621
2,RED CROSSER,11334,67.503441,0.026572,1.544366
3,AGNES LIXARVE,35009,21.853923,0.137724,2.591441
4,EASY COPI,77718,9.844360,0.118195,1.001817
5,LIVI ON THE RUN,171023,4.473574,0.170248,0.655754
6,KING SIR RAYMOND,33484,22.849241,0.064745,1.273744
7,STEPPING FERRARI,170529,4.486533,0.122410,0.472859
8,GLOBAL WILDCAT,18950,40.373826,0.078494,2.728590
9,WISS ZORNITA,13150,58.181293,0.024017,1.203117


In [57]:
actual_gain=2820
betting=150 # on horse 3

In [64]:
df.loc[3].odds*betting*(1-track_take) 
# They are matching !!

2822.434191207975

In [67]:
programme =execute_get_query(url=get_pmu_api_url(url_name="PROGRAMME", date=dt.date(2020, 4,27)))
r_i=1
c_i=11

In [68]:
courses_ = [
    course
    for reunion in programme["programme"]["reunions"]
    for course in reunion["courses"]
    if course["numReunion"] == r_i and course["numOrdre"] == c_i
]
assert len(courses_) == 1
course = courses_[0]

In [69]:
course['statut']

'FIN_COURSE'

In [72]:
citations_inter = execute_get_query(
        url=get_pmu_api_url(url_name="CITATIONS", date=date, r_i=r_i, c_i=c_i)
        + "?specialisation=INTERNET"
    )

In [73]:
citations = execute_get_query(
        url=get_pmu_api_url(url_name="CITATIONS", date=date, r_i=r_i, c_i=c_i)
    )

In [80]:
[citation['typePari'] for citation in citations_inter['listeCitations']]

['E_COUPLE_GAGNANT',
 'E_TRIO',
 'E_COUPLE_PLACE',
 'E_DEUX_SUR_QUATRE',
 'E_SIMPLE_PLACE',
 'E_SIMPLE_GAGNANT',
 'E_MINI_MULTI']

In [81]:
[citation['typePari'] for citation in citations['listeCitations']]

KeyError: 'listeCitations'

In [82]:
citations

{'get_url': 'https://online.turfinfo.api.pmu.fr/rest/client/1/programme/27042020/R1/C11/citations',
 'note': 'no content',
 'query_date': '2020-04-29T06:34:24.245028'}

In [86]:
course.keys()

dict_keys(['cached', 'arriveeDefinitive', 'departImminent', 'timezoneOffset', 'numReunion', 'numOrdre', 'numExterne', 'heureDepart', 'libelle', 'libelleCourt', 'montantPrix', 'parcours', 'distance', 'distanceUnit', 'discipline', 'specialite', 'categorieParticularite', 'conditionSexe', 'nombreDeclaresPartants', 'grandPrixNationalTrot', 'numSocieteMere', 'pariMultiCourses', 'pariSpecial', 'montantTotalOffert', 'montantOffert1er', 'montantOffert2eme', 'montantOffert3eme', 'montantOffert4eme', 'montantOffert5eme', 'conditions', 'numCourseDedoublee', 'paris', 'statut', 'categorieStatut', 'dureeCourse', 'participants', 'ecuries', 'rapportsDefinitifsDisponibles', 'isArriveeDefinitive', 'isDepartImminent', 'isDepartAJPlusUn', 'cagnottes', 'pronosticsExpires', 'replayDisponible', 'hippodrome', 'epcPourTousParis', 'courseTrackee', 'formuleChampLibreIndisponible', 'hasEParis', 'ordreArrivee'])

In [91]:
dt.date.fromtimestamp(1587938400000/1000)

datetime.date(2020, 4, 27)

In [92]:
programme['programme']['date']

1587938400000

In [105]:
import requests

date=dt.date.today()
r_i=1
c_i=1
requests.get(f"https://online.turfinfo.api.pmu.fr/rest/client/1/programme/{date.strftime('%d%m%Y')}/R{r_i}/C{c_i}/citations?specialisation=OFFLINE")

<Response [403]>